# pg_server_cache 
Looking on the cache of a PG Server


## Configuration
Open a connection to the DB. Use a Connection String stored in a .cfg file

In [1]:
import sqlalchemy
import pandas as pd
import configparser
import matplotlib.pyplot as plt 

# Read from the Config file
config = configparser.ConfigParser() 
config.read_file(open(r'../ipynb.cfg'))

con_str = config.get('con_str', 'PG_AIRBASES') 

engine = sqlalchemy.create_engine(con_str)

# print("Connecting with engine " + str(engine))
try:
    connection = engine.connect()
except (Exception, sqlalchemy.exc.SQLAlchemyError) as error:
    print("Error while connecting to PostgreSQL database:", error)



## Pre-Req
Check whether the extension exists
TODO: The cell below doesn't really work. The query returns "false" but the code ignores it. Maybe it returns the string "false" rather than a bit

In [2]:

from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError

sql_command = """
SELECT CASE WHEN COUNT(*) > 0 THEN B'1'::BIT(1) ELSE B'0'::BIT(1) END AS extension_exists
FROM pg_extension
WHERE extname = 'pg_buffercache';
"""
df = pd.read_sql_query(sql_command, connection)

try:
    # Execute the SQL command
   
    result = connection.execute(sql_command).fetchone()

    # print (type(result[0]))
        # Check the result
    if result[0] == '0':
        raise ValueError("Error: The extension pg_buffercache doesn't exist ")
    else: 
        print ("The Extension pg_buffercache exists")
except (SQLAlchemyError, ValueError) as e:
    # Handle any errors or raised exceptions
    raise e


The Extension pg_buffercache exists


## BUFFER CACHE
Postgres caches recent queries in memory called the shared buffer cache (shared_buffers in postgresql.conf). The pg_statio_user_tables has as rows representing various stats on each of the (user) tables. The two columns of interest we’ll be looking at are:

* pg_statio_user_tables.heap_blks_read — Number of disk blocks read from a table (ie. missed cache)
* pg_statio_user_tables.heap_blks_hits — Number of buffer hits from this table (ie. cache hit)


In [3]:
query_ccnnections_metrics = """
SELECT 
  schemaname, 
  pg_class.relname, 
  ROUND(
    CASE 
      WHEN heap_blks_hit + heap_blks_read = 0 
      THEN 0 
      ELSE heap_blks_hit::decimal / (heap_blks_hit + heap_blks_read) 
    END, 
    3
  ) as cache_hit_ratio
FROM 
  pg_statio_user_tables 
  JOIN pg_class ON pg_statio_user_tables.relid = pg_class.oid 
  JOIN pg_namespace ON pg_namespace.oid = pg_class.relnamespace;
"""

df = pd.read_sql_query(query_ccnnections_metrics, connection)
df

,schemaname,relname,cache_hit_ratio
0,metis,pg_stat_database_snapshots,0.886
1,metis,queries,0.997
2,metis,pg_stat_tables_activity_snapshots,0.917
3,cron,job,0.579
4,logs,postgres_logs,0.000
5,public,sales_table,0.991
6,metis,postgres_server_settings,0.000
7,metis_qa,test_high_indexes_num,0.000
8,postgres_air,aircraft,0.000
9,postgres_air,airport,0.773
